# Import everything


In [ ]:
from typing import Tuple
from os.path import exists
import torch
import numpy as np
from tqdm.notebook import tqdm
from torch.utils.data import DataLoader
from settings import cfg
from helper import loader
import export_result


# Load everything

In [2]:
config = cfg.TrainConfig()


Param size: 512.227MB
Buffer size: 0.032MB
Stats:
        | Number of not-punching: 34006
        | Number of punching: 6355


# Train

In [12]:
from sklearn.metrics import f1_score


def train(train_idx: np.ndarray) -> 'Tuple(float, float)':
    config.model.train()
    train_dataloader = config.get_dataloader(train_idx)
    total_loss_train = 0

    prediction_array = []
    label_array = []
    for image, label in tqdm(train_dataloader):
        loader.to_device([image, label], config.device)
        image = config.transforms(image)

        output = config.model(image)
        batch_loss = config.criterion(output, label)
        total_loss_train += batch_loss.item()

        prediction = output.argmax(dim=1)
        prediction_array.append(prediction.cpu().numpy())
        label_array.append(label.cpu().numpy())

        config.optimizer.zero_grad()
        batch_loss.backward()
        config.optimizer.step()
        config.model.zero_grad()

    prediction_array = np.concatenate(prediction_array)
    label_array = np.concatenate(label_array)

    total_accumulate_train = (prediction_array == label_array).sum().item()
    f1_score_train = f1_score(label_array, prediction_array, average='macro')

    return total_loss_train, total_accumulate_train, f1_score_train


In [13]:
def judge(judge_idx: np.ndarray) -> 'Tuple(float, float)':
    config.model.eval()
    judge_dataloader = config.get_dataloader(judge_idx)
    total_loss_judge = 0

    prediction_array = []
    label_array = []
    with torch.no_grad():
        for image, label in tqdm(judge_dataloader):
            loader.to_device([image, label], config.device)

            output = config.model(image)
            batch_loss = config.criterion(output, label)
            total_loss_judge += batch_loss.item()

            prediction = output.argmax(dim=1)
            prediction_array.append(prediction.cpu().numpy())
            label_array.append(label.cpu().numpy())

    prediction_array = np.concatenate(prediction_array)
    label_array = np.concatenate(label_array)

    total_accumulate_judge = (prediction_array == label_array).sum().item()
    f1_score_judge = f1_score(label_array, prediction_array, average='macro')

    return total_loss_judge, total_accumulate_judge, f1_score_judge

In [14]:
min_judge_loss = float('inf')
last_submit = 0

print("Starting phase 1")

for epoch, (train_idx, judge_idx) in enumerate(config.get_split()):
    print(f'''Starting epoch {epoch+1}
        | Train size: {train_idx.shape[0]}
        | Judge size: {judge_idx.shape[0]}''')
    total_loss_train, total_accumulate_train, f1_score_train = train(train_idx)
    total_loss_judge, total_accumulate_judge, f1_score_judge = judge(judge_idx)

    print(
        f'''Epoch: {epoch+1} 
        | Train Loss: {total_loss_train/len(train_idx):.3f}
        | Train Accuracy: {total_accumulate_train/len(train_idx):.3f}
        | Train F1 Score: {f1_score_train:.3f}
        | Val Loss: {total_loss_judge/len(judge_idx):.3f}
        | Val F1 Score: {f1_score_judge:.3f}''')
    
    if last_submit == 0:
        last_submit = f1_score_judge
    elif f1_score_judge - last_submit > 0.05:
        last_submit = f1_score_judge
        export_result.submit(config)

    if min_judge_loss > total_loss_judge/len(judge_idx):
        min_judge_loss = total_loss_judge/len(judge_idx)
        config.save_checkpoint()
        print(
            f"Save model because val loss improve loss {min_judge_loss:.3f}")
        under_min = 0
    else:
        under_min += 1
        if under_min > cfg.early_stop:
            print(
                f"Early stop because val loss not improve {under_min} epochs. Resetting model.")
            config.load_best()
            under_min = 0


Starting phase 1
Starting epoch 1
        | Train size: 36324
        | Judge size: 4037


  0%|          | 0/1136 [00:00<?, ?it/s]

cuda
torch.Size([32, 3, 224, 224]) <class 'torch.Tensor'> torch.float32 cpu


RuntimeError: Input type (torch.FloatTensor) and weight type (torch.cuda.FloatTensor) should be the same or input should be a MKLDNN tensor and weight is a dense tensor